## RECURRENT NEURAL NETWORKS USING LSTMS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing The Required Libraries

In [ ]:
import pandas as pd;
import numpy as np;
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

Loading The Preprocessed Dataset

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/RNN_Preprocessed_Dataset.csv")

Split Into Train & Test

In [ ]:
X_train = []
Y_train = []
for i in range(40,len(dataset)):
    X_train.append(dataset.iloc[i-40:i])
    Y_train.append(dataset.iloc[i])
X_train,Y_train = np.array(X_train), np.array(Y_train)

Building The Neural Network

In [ ]:
rnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2])),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1),
])

In [ ]:
rnn.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),loss="mean_squared_error")

Callback Function

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(patience=3,monitor="loss")

In [ ]:
rnn.summary()

In [ ]:
rnn.fit(X_train,Y_train,epochs=100,batch_size=32,callbacks=[callback])